# Preparación Datos

In [1]:
# Data handling
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)


import glob
import json

# Data visualization
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import missingno as mso
import matplotlib.ticker as ticker
import geopandas as gpd

import ydata_profiling as pp
from sklearn import cluster, mixture
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import kneighbors_graph
from itertools import cycle, islice
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px

# Stats
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro, norm
# warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Lee el archivo CSV en un DataFrame
df = pd.read_csv('dataset_completo.csv')

In [3]:
# Convierte la columna que contiene la fecha en un formato de fecha
df['Date'] = pd.to_datetime(df['DataAccadimento'])

# Extrae el año, mes, día de la semana y día del mes de la fecha
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day_of_week'] = df['Date'].dt.day_name()
df['Day_of_month'] = df['Date'].dt.day
df.head()

,DataRilevazione,DataProtocollo,DataAccadimento,DataDefinizione,DataMorte,LuogoAccadimento,IdentificativoInfortunato,Genere,Eta,LuogoNascita,ModalitaAccadimento,ConSenzaMezzoTrasporto,IdentificativoCaso,DefinizioneAmministrativa,DefinizioneAmministrativaEsitoMortale,Indennizzo,DecisioneIstruttoriaEsitoMortale,GradoMenomazione,GiorniIndennizzati,IdentificativoDatoreLavoro,PosizioneAssicurativaTerritoriale,SettoreAttivitaEconomica,Gestione,GestioneTariffaria,GrandeGruppoTariffario,Date,Year,Month,Day_of_week,Day_of_month
0,30/04/2023,18/06/2018,17/06/2018,28/06/2018,NaN,69,3895603,M,41,ITAL,N,N,21619890,P,ND,TE,ND,-1,10,9270978,11500123,H 52,I,1,9,2018-06-17,2018,6,Sunday,17
1,30/04/2023,04/10/2021,30/09/2021,27/10/2021,NaN,66,3882671,M,61,ITAL,N,N,23839687,N,ND,NE,ND,-1,0,-1,-1,ND,A,ND,ND,2021-09-30,2021,9,Thursday,30
2,30/04/2023,28/09/2020,23/09/2020,14/11/2020,NaN,66,3897622,M,52,ITAL,N,N,23119539,P,ND,TE,ND,-1,45,3325574,4350088,H 49,I,1,9,2020-09-23,2020,9,Wednesday,23
3,30/04/2023,06/04/2022,04/04/2022,22/04/2022,NaN,69,3896347,F,49,ITAL,N,N,24228399,P,ND,TE,ND,-1,12,4173909,13125854,Q 86,I,4,0,2022-04-04,2022,4,Monday,4
4,30/04/2023,21/02/2020,19/02/2020,17/03/2020,NaN,68,3894028,M,69,ITAL,N,N,22815467,P,ND,TE,ND,-1,26,-1,-1,ND,A,ND,ND,2020-02-19,2020,2,Wednesday,19


In [4]:
df.columns = ['FechaEncuesta', 'FechaRegistro', 'FechaAccidente', 'FechaDefiniciónAdmin', 'FechaMuerte', 'Provincia', 'CódigoInterno',
              'Género', 'Edad', 'LugarNacimiento', 'ModalidadAccidente', 'ImplicaciónMedioTransporte', 'IdentificativoCaso', 'SituaciónAdmin',
              'DefAdminEvidienciaMortal', 'Indemnización', 'DecisiónInstrucciónResultadoMortal', 'GradoDeterioro', 'DíasIndemnizados', 
              'IdentificadorEmpleador', 'PosiciónAseguramientoTerritorial', 'DetalleActividadEconómica', 'Gestión', 'TipoActividadEconómica', 
              'SectorEconómico', 'Fecha', 'Año', 'Mes', 'DiaSemana', 'DiaMes']

In [5]:
conteo_valores = df['GradoDeterioro'].value_counts()

# Imprimir el resultado
print('Número de veces que aparece cada valor único en GradoDeterioro:')
print(conteo_valores)

Número de veces que aparece cada valor único en GradoDeterioro:
GradoDeterioro
-1     2802570
 2       63394
 3       47000
 6       43615
 4       39823
        ...   
 98          2
 76          2
 81          2
 99          1
 83          1
Name: count, Length: 99, dtype: int64


#Aparecen muchas veces el valor -1 en la variable GradoDeterioro. Vamos a eliminar estos valores ya que no aportan informción.

In [6]:
df = df[df['GradoDeterioro'] != -1]
df.shape

(326877, 30)

In [7]:
conteo_edad_menos_uno = df['Edad'].value_counts().get(-1, 0)

print(f"La cantidad de veces que 'Edad' es igual a -1 es: {conteo_edad_menos_uno}")

La cantidad de veces que 'Edad' es igual a -1 es: 11


In [8]:
df = df[df['Edad'] >= 16]

In [9]:
media_grado_deterioro = df['GradoDeterioro'].mean()

print(f"La media de 'GradoDeterioro' es: {media_grado_deterioro}")

La media de 'GradoDeterioro' es: 6.707736071430775


In [10]:
territorial_codes = pd.read_csv('Codici-statistici-e-denominazioni.csv', sep=";", encoding = "ISO-8859-1")


In [11]:
territorial_codes.columns = ['region code', 'Code of the supra-municipal territorial unit', 'Province Code (Historical)', 'Municipality progressive',
                            'Municipality code in alphanumeric format', 'Name (Italian and foreign)', 'Name in Italian', 'Name in other language',
                            'Code Geographical Distribution', 'Geographical distribution', 'Nombre Región', 'Nombre Unidad Territorial',
                            'Type of supra-municipal territorial unit', 'Flag Provincial capital / metropolitan city / free consortium',
                            'Automotive abbreviation', 'Municipal code numeric format', 'Numeric municipality code with 110 provinces (from 2010 to 2016)',
                            'Numeric municipality code with 107 provinces (from 2006 to 2009)', 'Numeric municipality code with 103 provinces (1995 to 2005)',
                            'Cadastral code of the municipality', 'NUTS1 code 2010', 'NUTS2 code 2010', 'NUTS3 code 2010', 'NUTS1 code 2021', 'NUTS2 code 2021',
                            'NUTS3 code 2021']

In [12]:
code_province_name_region = pd.concat([territorial_codes['Province Code (Historical)'],
                                     territorial_codes['Automotive abbreviation'],
                                     territorial_codes['Nombre Región'],
                                      territorial_codes['Nombre Unidad Territorial']], axis = 1)

In [13]:
# When the province is Naples, the abbreviation is NA. This is read as a missing value, replace NA with 'NAPOLI' in full:
code_province_name_region['Automotive abbreviation'][code_province_name_region['Automotive abbreviation'].isna()] = 'NAPOLI' 
code_province_name_region.columns = ['PlaceOfOccurrence', 'abbreviation', 'Region', 'Unidad Territorial' ]

code_province_name_region = code_province_name_region.drop_duplicates()
code_province_name_region.head()

,PlaceOfOccurrence,abbreviation,Region,Unidad Territorial
0,1,TO,Piemonte,Torino
312,2,VC,Piemonte,Vercelli
394,3,NO,Piemonte,Novara
481,4,CN,Piemonte,Cuneo
728,5,AT,Piemonte,Asti


In [14]:
# merge accident data with ISTAT data
data = pd.merge(df, code_province_name_region, left_on='Provincia', right_on='PlaceOfOccurrence', how='left')


In [15]:
data_TipoActividadEconómica = pd.read_csv("GestioneTariffaria.csv", sep = ";", encoding = "ISO-8859-1")


In [16]:
data_TipoActividadEconómica = pd.read_csv("GestioneTariffaria.csv", sep = ";", encoding = "ISO-8859-1")


In [17]:
data_TipoActividadEconómica.columns = ["TipoActividadEconómica", "DescrTipoActividadEconómica"]
data1 = pd.merge(data, data_TipoActividadEconómica, on='TipoActividadEconómica', how='left')

In [18]:
data_DetalleActividadEconómica = pd.read_csv("SettoreAttivitaEconomica.csv", sep = ";", encoding = "ISO-8859-1")

In [19]:
data2 = pd.merge(data1, data_DetalleActividadEconómica, left_on='DetalleActividadEconómica', right_on='SettoreAttivitaEconomica', how='left')

In [20]:
data_SectorEconómico = pd.read_csv("GrandeGruppoTariffario.csv", sep = ";", encoding = "ISO-8859-1")

In [21]:
data_SectorEconómico.columns = ["SectorEconómico", "DescrSectorEconómico"]
data3 = pd.merge(data2, data_SectorEconómico, on='SectorEconómico', how='left')

In [22]:
conteo_SectorX = data3['DescrSectorEconómico'].value_counts()

# Imprimir el resultado
print('Número de veces que aparece cada valor único en DescrSectorEconómico:')
print(conteo_SectorX)

Número de veces que aparece cada valor único en DescrSectorEconómico:
DescrSectorEconómico
GG0 Attività varie              122148
X Non determinato                61187
GG3 Costruzioni e impianti       43869
GG6 Metalli e macchinari         35839
GG9 Trasporti e magazzini        26028
GG1 Lav. agricole e alimenti     10565
GG2 Chimica, carta e cuoi         8747
GG5 Legno e affini                5617
GG7 Mineraria, rocce e vetro      4445
GG8 Tessile e confezioni          4328
GG4 Energia e comunicazioni       1345
Name: count, dtype: int64


In [23]:
data3 = data3[data3['DescrSectorEconómico'] != 'X Non determinato']
data3.shape

(262931, 40)

In [24]:
data3 = data3.fillna(0)

In [25]:
data3['Muerte'] = data3['FechaMuerte'].apply(lambda fecha: 1 if fecha != 0 else 0)

In [26]:
DescrTipoActividadEconómica_ = {'Non determinata': 'No Determinada',
                           'Industria':'Industria',
                           'Altre Attività':'Otra Actividad',
                           'Artigianato': 'Artesanía',
                           'Terziario':'Terciario'}
                           
data3['DescrTipoActividadEconómica']=data3['DescrTipoActividadEconómica'].map(DescrTipoActividadEconómica_)

In [27]:
# Diccionario de mapeo para 'SectorEconómico'
DescrSectorEconómico_ = {'GG9 Trasporti e magazzini': 'Transporte y almacenamiento', 
                         'GG0 Attività varie': 'Actividades diversas', 'GG6 Metalli e macchinari': 'Metales y maquinaria', 
                         'GG3 Costruzioni e impianti': 'Construcción e instalaciones', 'GG1 Lav. agricole e alimenti': 'Agricultura y alimentación', 
                         'GG8 Tessile e confezioni': 'Textil y confección', 'GG2 Chimica, carta e cuoi': 'Química, papel y cuero', 
                         'GG5 Legno e affini': 'Madera y afines', 'GG7 Mineraria, rocce e vetro': 'Minería, rocas y vidrio', 
                         'GG4 Energia e comunicazioni': 'Energía y comunicaciones'}

# Aplicar el mapeo a la columna 'SectorEconómico'
data3['DescrSectorEconómico'] = data3['DescrSectorEconómico'].map(DescrSectorEconómico_)

In [28]:
Gestión_ = {'I':1,
           'A':2,
           'S':3}
data3['Gestión']=data3['Gestión'].map(Gestión_)

In [29]:
TipoActividadEconómica_ = {'ND':0,
                           '1':1,
                           '2':2,
                           '3':3,
                           '4':4}
                           
data3['TipoActividadEconómica']=data3['TipoActividadEconómica'].map(TipoActividadEconómica_)

In [30]:
# Diccionario de mapeo para 'SectorEconómico'
SectorEconómico_ = {'ND': 10, '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

# Aplicar el mapeo a la columna 'SectorEconómico'
data3['SectorEconómico'] = data3['SectorEconómico'].map(SectorEconómico_)

In [31]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    (1, 5): 1,
    (6, 15): 2,
    (16, 25): 3,
    (26, 50): 4,
    (51, 85): 5,
    (86, 100): 6
}

data3['GradoDeterioro'] = pd.cut(data3['GradoDeterioro'], bins=[0, 5, 15, 25, 50, 85, 100], labels=list(intervalos.values()), include_lowest=True)
# Transformar la columna 'GradoDeterioro' a tipo int
data3['GradoDeterioro'] = data3['GradoDeterioro'].astype(int)

In [32]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    (16, 25): 1,
    (26, 35): 2,
    (36, 45): 3,
    (46, 55): 4,
    (56, 100): 5  # Ajusté el límite superior a 56 para que coincida con el diccionario
}

data3['Edad'] = pd.cut(data3['Edad'], bins=[16, 26, 36, 46, 56, 100], labels=list(intervalos.values()), include_lowest=True)

# Transformar la columna 'Edad' a tipo int
data3['Edad'] = data3['Edad'].astype(int)


In [33]:
# Seleccionar las columnas que deseas conservar
columns_to_keep = ['Género', 'Edad', 'ModalidadAccidente', 
                   'ImplicaciónMedioTransporte', 'GradoDeterioro', 
                   'SectorEconómico', 'Año', 'Mes', 'Muerte', 'Region']

# Crear un nuevo DataFrame solo con las columnas seleccionadas
dataa = data3[columns_to_keep]

In [34]:

# Crear variables dummy para 'ModalidadAccidente' y 'ImplicaciónMedioTransporte'
ModalidadAccidente_dummy = pd.get_dummies(dataa['ModalidadAccidente'], prefix='CaminoTrabajo')
ImplicaciónMedioTransporte_dummy = pd.get_dummies(dataa['ImplicaciónMedioTransporte'], prefix='ImplicaciónMedioTransporte')
Género_dummy = pd.get_dummies(dataa['Género'], prefix='Género')

# Convertir las columnas dummy a valores binarios (0 o 1)
ModalidadAccidente_dummy = ModalidadAccidente_dummy.astype(int)
ImplicaciónMedioTransporte_dummy = ImplicaciónMedioTransporte_dummy.astype(int)
Género_dummy = Género_dummy.astype(int)

# Concatenar las nuevas columnas dummy al DataFrame original
dataa = pd.concat([dataa, ModalidadAccidente_dummy, ImplicaciónMedioTransporte_dummy, Género_dummy], axis=1)


In [35]:
dataa=dataa.drop(['Género', 'Género_F', 'ImplicaciónMedioTransporte', 'ImplicaciónMedioTransporte_N', 'ModalidadAccidente', 'CaminoTrabajo_N'], axis=1)


In [36]:
regiones = {'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 
            'Liguria', 'Lombardia', 'Marche', 'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 
            'Trentino-Alto Adige/Südtirol', 'Umbria', "Valle d'Aosta/Vallée d'Aoste", 'Veneto'}

divisiones = {
    'Noroccidental': {'Liguria', 'Lombardia', 'Piemonte', "Valle d'Aosta/Vallée d'Aoste"},
    'Nororiental': {'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Trentino-Alto Adige/Südtirol', 'Veneto'},
    'Central': {'Lazio', 'Marche', 'Toscana', 'Umbria'},
    'Meridional': {'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Molise', 'Puglia'},
    'Insular': {'Sardegna', 'Sicilia'}
}

# Mapear cada región a su división correspondiente
mapeo_divisiones = {}
for region in regiones:
    for division, regiones_division in divisiones.items():
        if region in regiones_division:
            mapeo_divisiones[region] = division
            break

dataa['Region'] = dataa['Region'].map(mapeo_divisiones)


In [37]:
Region_ = {'Noroccidental':4, 
           'Nororiental':3, 
           'Central':2, 
           'Meridional':0,
           'Insular':1}
                           
dataa['Region']=dataa['Region'].map(Region_)

In [38]:
dataa.head()

,Edad,GradoDeterioro,SectorEconómico,Año,Mes,Muerte,Region,CaminoTrabajo_S,ImplicaciónMedioTransporte_S,Género_M
0,4,1,3,2018,5,0,0,0,0,1
1,5,2,0,2018,5,0,0,1,0,0
2,5,2,0,2021,4,0,0,0,0,1
5,5,3,3,2022,9,0,0,1,1,1
8,5,1,0,2021,9,0,0,0,1,0


In [71]:
data=dataa.copy()

In [73]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 14 clústeres
kmeans_S_1 = KMeans(n_clusters=14, random_state=42)
data['Cluster_S_1'] = kmeans_S_1.fit_predict(subset_data)
# Calcular la inercia
inertia_S_1 = kmeans_S_1.inertia_
# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_1}")

Inercia del modelo: 122080.81586994034


In [73]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 15 clústeres
kmeans_CH_1 = KMeans(n_clusters=15, random_state=42)
data['Cluster_CH_1'] = kmeans_CH_1.fit_predict(subset_data)
# Calcular la inercia
inertia_CH_1 = kmeans_CH_1.inertia_
# Imprimir la inercia
print(f"Inercia del modelo: {inertia_CH_1}")

Inercia del modelo: 122080.81586994034


In [74]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['SectorEconómico', 'Region', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 15 clústeres
kmeans_S_2 = KMeans(n_clusters=15, random_state=42)
data['Cluster_S_2'] = kmeans_S_2.fit_predict(subset_data)
# Calcular la inercia
inertia_S_2 = kmeans_S_2.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_2}")

Inercia del modelo: 155492.23898883146


In [75]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['SectorEconómico', 'Region', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 12 clústeres
kmeans_DB_2 = KMeans(n_clusters=12, random_state=42)
data['Cluster_DB_2'] = kmeans_DB_2.fit_predict(subset_data)
# Calcular la inercia
inertia_DB_2 = kmeans_DB_2.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_DB_2}")

Inercia del modelo: 192259.3017671872


In [76]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['SectorEconómico', 'Region', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 14 clústeres
kmeans_CH_2 = KMeans(n_clusters=14, random_state=42)
data['Cluster_CH_2'] = kmeans_CH_2.fit_predict(subset_data)
# Calcular la inercia
inertia_CH_2 = kmeans_CH_2.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_CH_2}")

Inercia del modelo: 174236.73109817496


In [77]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['Edad', 'GradoDeterioro', 'CaminoTrabajo_S', 'ImplicaciónMedioTransporte_S', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 15 clústeres
kmeans_S_3 = KMeans(n_clusters=15, random_state=42)
data['Cluster_S_3'] = kmeans_S_3.fit_predict(subset_data)
# Calcular la inercia
inertia_S_3 = kmeans_S_3.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_3}")

Inercia del modelo: 104277.93236147099


In [79]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'Region', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 14 clústeres
kmeans_S_4 = KMeans(n_clusters=14, random_state=42)
data['Cluster_S_4'] = kmeans_S_4.fit_predict(subset_data)
# Calcular la inercia
inertia_S_4 = kmeans_S_4.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_4}")

Inercia del modelo: 235484.52700772244


In [78]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'Region', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 4 clústeres
kmeans_CH_4 = KMeans(n_clusters=4, random_state=42)
data['Cluster_CH_4'] = kmeans_CH_4.fit_predict(subset_data)
# Calcular la inercia
inertia_CH_4 = kmeans_CH_4.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_CH_4}")

Inercia del modelo: 687985.3636345738


In [81]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 14 clústeres
kmeans_S_5 = KMeans(n_clusters=14, random_state=42)
data['Cluster_S_5'] = kmeans_S_5.fit_predict(subset_data)
# Calcular la inercia
inertia_S_5 = kmeans_S_5.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_5}")

Inercia del modelo: 219450.5743534584


In [82]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 13 clústeres
kmeans_DB_5 = KMeans(n_clusters=13, random_state=42)
data['Cluster_DB_5'] = kmeans_DB_5.fit_predict(subset_data)
# Calcular la inercia
inertia_DB_5 = kmeans_DB_5.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_DB_5}")

Inercia del modelo: 224185.83820365014


In [83]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 6 clústeres
kmeans_CH_5 = KMeans(n_clusters=6, random_state=42)
data['Cluster_CH_5'] = kmeans_CH_5.fit_predict(subset_data)
# Calcular la inercia
inertia_CH_5 = kmeans_CH_5.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_CH_5}")

Inercia del modelo: 388955.5142728289


In [84]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 15 clústeres
kmeans_S_6 = KMeans(n_clusters=15, random_state=42)
data['Cluster_S_6'] = kmeans_S_6.fit_predict(subset_data)
# Calcular la inercia
inertia_S_6 = kmeans_S_6.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_S_6}")

Inercia del modelo: 224531.30195601957


In [85]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 13 clústeres
kmeans_DB_6 = KMeans(n_clusters=13, random_state=42)
data['Cluster_DB_6'] = kmeans_DB_6.fit_predict(subset_data)
# Calcular la inercia
inertia_DB_6 = kmeans_DB_6.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_DB_6}")

Inercia del modelo: 246844.00665681073


In [86]:
# Seleccionar solo las columnas necesarias del DataFrame
variables_to_cluster = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género_M']
subset_data = data[variables_to_cluster]

# Aplicar el algoritmo KMeans con 7 clústeres
kmeans_CH_6 = KMeans(n_clusters=7, random_state=42)
data['Cluster_CH_6'] = kmeans_CH_6.fit_predict(subset_data)
# Calcular la inercia
inertia_CH_6 = kmeans_CH_6.inertia_

# Imprimir la inercia
print(f"Inercia del modelo: {inertia_CH_6}")

Inercia del modelo: 389458.10657372855


In [87]:
data.head()

,Edad,GradoDeterioro,SectorEconómico,Año,Mes,Muerte,Region,CaminoTrabajo_S,ImplicaciónMedioTransporte_S,Género_M,Cluster_I_1,Cluster_S_1,Cluster_S_2,Cluster_DB_2,Cluster_CH_2,Cluster_I_3,Cluster_CH_4,Cluster_S_4,Cluster_GMM_4,Cluster_S_5,Cluster_DB_5,Cluster_CH_5,Cluster_S_6,Cluster_DB_6,Cluster_CH_6
0,4,1,3,2018,5,0,0,0,0,1,10,10,2,8,9,5,0,7,8,10,1,0,0,10,2
1,5,2,0,2018,5,0,0,1,0,0,4,11,4,3,11,4,1,11,3,11,8,1,4,4,3
2,5,2,0,2021,4,0,0,0,0,1,11,1,4,3,13,9,1,6,13,11,8,1,1,8,0
5,5,3,3,2022,9,0,0,1,1,1,13,2,2,8,9,7,0,7,8,2,12,0,6,0,5
8,5,1,0,2021,9,0,0,0,1,0,0,6,4,3,11,4,1,11,3,11,8,1,11,7,0


In [88]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    1: "1-5",
    2: "6-15",
    3: "16-25",
    4: "26-50",
    5: "51-85",
    6: "86-100"
}

data['GradoDeterioro']=data['GradoDeterioro'].map(intervalos)

In [89]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    1: "16-25",
    2: "26-35",
    3: "36-45",
    4: "46-55",
    5: "+56"}

data['Edad']=data['Edad'].map(intervalos)

In [90]:
Region_ = {4:'Noroccidental', 
           3:'Nororiental', 
           2:'Central', 
           0:'Meridional',
           1:'Insular'}
                           
data['Region']=data['Region'].map(Region_)

In [91]:
Género_M_ = { 1:'Hombre', 0:'Mujer'}
                           
data['Género_M']=data['Género_M'].map(Género_M_)

In [92]:
Muerte_ = { 1:'Sí', 0:'No'}
                           
data['Muerte']=data['Muerte'].map(Muerte_)

In [93]:
# Diccionario de mapeo para 'SectorEconómico'
SectorEconómico_ = {9: 'Transporte y almacenamiento', 
                         0: 'Actividades diversas', 6: 'Metales y maquinaria', 
                         3: 'Construcción e instalaciones', 1: 'Agricultura y alimentación', 
                         8: 'Textil y confección', 2: 'Química, papel y cuero', 
                         5: 'Madera y afines', 7: 'Minería, rocas y vidrio', 
                         4: 'Energía y comunicaciones'}

# Aplicar el mapeo a la columna 'SectorEconómico'
data['SectorEconómico'] = data['SectorEconómico'].map(SectorEconómico_)

In [94]:
CaminoTrabajo_S_ = { 1:'Sí', 0:'No'}
                           
data['CaminoTrabajo_S']=data['CaminoTrabajo_S'].map(CaminoTrabajo_S_)

In [95]:
ImplicaciónMedioTransporte_S_ = { 1:'Sí', 0:'No'}
                           
data['ImplicaciónMedioTransporte_S']=data['ImplicaciónMedioTransporte_S'].map(ImplicaciónMedioTransporte_S_)

In [96]:
data.head()

,Edad,GradoDeterioro,SectorEconómico,Año,Mes,Muerte,Region,CaminoTrabajo_S,ImplicaciónMedioTransporte_S,Género_M,Cluster_I_1,Cluster_S_1,Cluster_S_2,Cluster_DB_2,Cluster_CH_2,Cluster_I_3,Cluster_CH_4,Cluster_S_4,Cluster_GMM_4,Cluster_S_5,Cluster_DB_5,Cluster_CH_5,Cluster_S_6,Cluster_DB_6,Cluster_CH_6
0,46-55,1-5,Construcción e instalaciones,2018,5,No,Meridional,No,No,Hombre,10,10,2,8,9,5,0,7,8,10,1,0,0,10,2
1,+56,6-15,Actividades diversas,2018,5,No,Meridional,Sí,No,Mujer,4,11,4,3,11,4,1,11,3,11,8,1,4,4,3
2,+56,6-15,Actividades diversas,2021,4,No,Meridional,No,No,Hombre,11,1,4,3,13,9,1,6,13,11,8,1,1,8,0
5,+56,16-25,Construcción e instalaciones,2022,9,No,Meridional,Sí,Sí,Hombre,13,2,2,8,9,7,0,7,8,2,12,0,6,0,5
8,+56,1-5,Actividades diversas,2021,9,No,Meridional,No,Sí,Mujer,0,6,4,3,11,4,1,11,3,11,8,1,11,7,0


In [97]:
data.columns = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Año', 'Mes', 'Muerte', 'Región',
                'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género', 'Cluster_S_1', 'Cluster_CH_1', 'Cluster_S_2', 
                'Cluster_DB_2', 'Cluster_CH_2', 'Cluster_S_3', 'Cluster_CH_4', 'Cluster_S_4', 'Cluster_S_5', 'Cluster_DB_5', 'Cluster_CH_5',
                'Cluster_S_6', 'Cluster_DB_6', 'Cluster_CH_6',]

In [205]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_I_1'].unique():
    cluster_data = data[data['Cluster_I_1'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_I_1.xlsx')
print(frecuencia_porcentaje_cluster)


    Cluster              Variable                         Valor  Porcentaje  \
0        10        GradoDeterioro                           1-5  100.000000   
1        10       SectorEconómico  Construcción e instalaciones   77.398985   
2        10         CaminoTrabajo                            No   92.369855   
3        10  ImplicaciónMedTransp                            No   90.761135   
4        10                Género                        Hombre   96.620936   
..      ...                   ...                           ...         ...   
70        2        GradoDeterioro                           1-5   53.860820   
71        2       SectorEconómico   Transporte y almacenamiento   85.281220   
72        2         CaminoTrabajo                            Sí   56.224976   
73        2  ImplicaciónMedTransp                            Sí  100.000000   
74        2                Género                        Hombre   87.816969   

    PorcentajeCluster  
0           10.118624  
1  

In [59]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_1'].unique():
    cluster_data = data[data['Cluster_S_1'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_1.xlsx')


In [ ]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_1'].unique():
    cluster_data = data[data['Cluster_S_1'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_1.xlsx')


In [61]:
# Variables de interés
variables_interes = ['SectorEconómico', 'Región', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_2'].unique():
    cluster_data = data[data['Cluster_S_2'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_2.xlsx')


In [61]:
# Variables de interés
variables_interes = ['SectorEconómico', 'Región', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_DB_2'].unique():
    cluster_data = data[data['Cluster_DB_2'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_DB_2.xlsx')


In [62]:
# Variables de interés
variables_interes = ['SectorEconómico', 'Región', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_CH_2'].unique():
    cluster_data = data[data['Cluster_CH_2'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_CH_2.xlsx')


In [62]:
# Variables de interés
variables_interes = ['Edad', 'GradoDeterioro', 'CaminoTrabajo', 'ImplicaciónMedTransp', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_I_3'].unique():
    cluster_data = data[data['Cluster_I_3'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_I_3.xlsx')


In [174]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'Región', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_CH_4'].unique():
    cluster_data = data[data['Cluster_CH_4'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_CH_4.xlsx')


In [64]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'Región', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_4'].unique():
    cluster_data = data[data['Cluster_S_4'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_4.xlsx')


In [65]:
# Variables de interés
variables_interes = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_5'].unique():
    cluster_data = data[data['Cluster_S_5'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_5.xlsx')


In [240]:
# Variables de interés
variables_interes = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_DB_5'].unique():
    cluster_data = data[data['Cluster_DB_5'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_DB_5.xlsx')


In [241]:
# Variables de interés
variables_interes = ['Edad', 'GradoDeterioro', 'SectorEconómico', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_CH_5'].unique():
    cluster_data = data[data['Cluster_CH_5'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_CH_5.xlsx')


In [106]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_S_6'].unique():
    cluster_data = data[data['Cluster_S_6'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_S_6.xlsx')


In [242]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_DB_6'].unique():
    cluster_data = data[data['Cluster_DB_6'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_DB_6.xlsx')


In [243]:
# Variables de interés
variables_interes = ['GradoDeterioro', 'SectorEconómico', 'Año', 'Género']

# Crear la lista para almacenar los datos
data_rows = []

for cluster in data['Cluster_CH_6'].unique():
    cluster_data = data[data['Cluster_CH_6'] == cluster]
    
    # Calcular el porcentaje de datos que pertenecen a este cluster
    porcentaje_cluster = (cluster_data.shape[0] / data.shape[0]) * 100
    
    for column in variables_interes:
        valores_mas_frecuentes = cluster_data[column].value_counts().index[:2]  # Obtener los dos valores más frecuentes
        porcentaje1 = (cluster_data[column] == valores_mas_frecuentes[0]).mean() * 100
        data_rows.append({'Cluster': cluster, 'Variable': column, 'Valor': valores_mas_frecuentes[0], 'Porcentaje': porcentaje1, 'PorcentajeCluster': porcentaje_cluster})

# Crear la DataFrame final
frecuencia_porcentaje_cluster = pd.DataFrame(data_rows)

# Imprimir la tabla de frecuencia por cluster
frecuencia_porcentaje_cluster.to_excel('frecuencia_CH_6.xlsx')
